<a href="https://colab.research.google.com/github/Phenwathe-alt/Msc-Project-Portfolio/blob/main/Project_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import gradio as gr
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pvlib.solarposition import get_solarposition
import joblib
import matplotlib.pyplot as plt

# Fetch weather data from OpenWeatherMap
def fetch_weather_data(lat, lon):
    api_key = "97df6b97ff5b701ffb90b2826a1b67d8"  # Replace with your OpenWeatherMap API key
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url).json()

    if response.get("cod") != 200:
        return None, f"Error fetching weather data: {response.get('message', 'Unknown error')}"

    air_temp = response['main']['temp'] - 273.15  # Convert from Kelvin to Celsius
    relative_humidity = response['main']['humidity']
    cloud_opacity = response['clouds']['all']
    surface_pressure = response['main']['pressure']

    return {
        "air_temp": air_temp,
        "relative_humidity": relative_humidity,
        "cloud_opacity": cloud_opacity,
        "surface_pressure": surface_pressure,
    }, None

# Calculate zenith angle dynamically
def calculate_zenith_angle(lat, lon):
    times = pd.date_range(datetime.now(), periods=1, freq='D')
    solpos = get_solarposition(times, lat, lon)
    return solpos['apparent_zenith'].iloc[0]

# Train or Load the XGBoost model
def train_or_load_xgboost():
    try:
        # Attempt to load the pre-trained model
        model = joblib.load("xgb_model.pkl")
        print("Model loaded successfully.")
    except FileNotFoundError:
        print("No pre-trained model found. Training a new model...")
        # Load dataset
        data = pd.read_csv('solcast_dataset.csv', index_col='period_end', parse_dates=True)
        selected_features = ['air_temp', 'dhi', 'dni', 'cloud_opacity', 'relative_humidity', 'surface_pressure', 'zenith']
        target_variable = "gti"

        X = data[selected_features]
        y = data[target_variable]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the XGBoost model
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
        print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
        print(f"R²: {r2_score(y_test, y_pred):.2f}")

        # Save the trained model
        joblib.dump(model, "xgb_model.pkl")
        print("Model trained and saved successfully.")

    return model

# Initialize the model
xgb_model = train_or_load_xgboost()

# Multi-Step Forecasting for 5 Days
def xgb_forecast_5_days(lat, lon):
    """
    Forecast the next 5 days of solar irradiance using XGBoost with automated feature fetching.
    """
    try:
        # Fetch weather data
        weather_data, weather_error = fetch_weather_data(lat, lon)
        if weather_error:
            print(f"Weather API Error: {weather_error}")
            return pd.DataFrame({"Error": [weather_error]}), None

        # Calculate zenith angle
        zenith_angle = calculate_zenith_angle(lat, lon)
        print(f"Zenith Angle: {zenith_angle}")

        forecasts = []
        confidence_intervals = []
        dates = [(datetime.now() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 6)]

        for day in range(5):
            dhi = 100 + day * 10  # Example dynamic DHI
            dni = 300 + day * 15  # Example dynamic DNI

            input_features = [
                weather_data["air_temp"],
                dhi,
                dni,
                weather_data["cloud_opacity"],
                weather_data["relative_humidity"],
                weather_data["surface_pressure"],
                zenith_angle
            ]
            input_features = np.array([input_features])
            prediction = xgb_model.predict(input_features)[0]
            prediction = np.clip(prediction, 0, 1200)
            forecasts.append(prediction)

            # Adding a dummy confidence interval (replace with actual computation if needed)
            confidence_intervals.append((prediction - 50, prediction + 50))

        # Create a plot of the forecast
        plt.figure(figsize=(10, 5))
        plt.plot(dates, forecasts, marker='o', label="Forecasted Irradiance")

        # Add confidence intervals
        lower_bounds, upper_bounds = zip(*confidence_intervals)
        plt.fill_between(dates, lower_bounds, upper_bounds, color='blue', alpha=0.2, label="Confidence Interval")

        # Annotate peaks
        for i, (date, value) in enumerate(zip(dates, forecasts)):
            plt.text(date, value + 5, f"{value:.1f} W/m²", ha='center', fontsize=9)

        plt.title("Daily Solar Irradiance Forecast (5 Days)")
        plt.xlabel("Date")
        plt.ylabel("Forecasted Irradiance (W/m²)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig("daily_forecast_plot.png")

        return pd.DataFrame({"Date": dates, "Forecasted Irradiance": forecasts}), "daily_forecast_plot.png"

    except Exception as e:
        print(f"Error in forecasting: {e}")
        return pd.DataFrame({"Error": [str(e)]}), None

# Gradio Interface
def gradio_forecast(lat, lon):
    forecast_df, plot_path = xgb_forecast_5_days(lat, lon)
    return forecast_df, plot_path

iface = gr.Interface(
    fn=gradio_forecast,
    inputs=[
        gr.Number(label="Latitude"),
        gr.Number(label="Longitude")
    ],
    outputs=[
        gr.Dataframe(headers=["Date", "Forecasted Irradiance"], label="5-Day Forecast"),
        gr.Image(type="filepath", label="Daily Forecast Plot")
    ],
    title="5-Day Solar Irradiance Forecasting",
    description="Provide your location to forecast solar irradiance for the next 5 days using real-time weather data."
)

iface.launch()


No pre-trained model found. Training a new model...
MAE: 8.82
RMSE: 22.09
R²: 0.99
Model trained and saved successfully.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://efbad87841f3273fde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
